# Assignment 1: Vector Database Creation and Retrieval
## Day 6 Session 2 - RAG Fundamentals

**OBJECTIVE:** Create a vector database from a folder of documents and implement basic retrieval functionality.

**LEARNING GOALS:**
- Understand document loading with SimpleDirectoryReader
- Learn vector store setup with LanceDB
- Implement vector index creation
- Perform semantic search and retrieval

**DATASET:** Use the data folder in `Day_6/session_2/data/` which contains multiple file types

**INSTRUCTIONS:**
1. Complete each function by replacing the TODO comments with actual implementation
2. Run each cell after completing the function to test it
3. The answers can be found in the existing notebooks in the `llamaindex_rag/` folder


In [1]:
# Import required libraries
import os
from pathlib import Path
from typing import List
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

print("✅ Libraries imported successfully!")

c:\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported successfully!


In [2]:
# Configure LlamaIndex Settings (Using OpenRouter - No OpenAI API Key needed)
def setup_llamaindex_settings():
    """
    Configure LlamaIndex with local embeddings and OpenRouter for LLM.
    This assignment focuses on vector database operations, so we'll use local embeddings only.
    """
    # Check for OpenRouter API key (for future use, not needed for this basic assignment)
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        print("ℹ️  OPENROUTER_API_KEY not found - that's OK for this assignment!")
        print("   This assignment only uses local embeddings for vector operations.")
    
    # Configure local embeddings (no API key required)
    Settings.embed_model = HuggingFaceEmbedding(
        model_name="BAAI/bge-small-en-v1.5",
        trust_remote_code=True
    )
    
    print("✅ LlamaIndex configured with local embeddings")
    print("   Using BAAI/bge-small-en-v1.5 for document embeddings")

# Setup the configuration
setup_llamaindex_settings()


ℹ️  OPENROUTER_API_KEY not found - that's OK for this assignment!
   This assignment only uses local embeddings for vector operations.


c:\python\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gengi\AppData\Local\llama_index\llama_index\Cache\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


✅ LlamaIndex configured with local embeddings
   Using BAAI/bge-small-en-v1.5 for document embeddings


## 1. Document Loading Function

Complete the function below to load documents from a folder using `SimpleDirectoryReader`.

**Note:** This assignment uses local embeddings only - no OpenAI API key required! We're configured to use OpenRouter for future LLM operations.


In [3]:
def load_documents_from_folder(folder_path: str):
    """
    Load documents from a folder using SimpleDirectoryReader.
    
    TODO: Complete this function to load documents from the given folder path.
    HINT: Use SimpleDirectoryReader with recursive parameter to load all files
    
    Args:
        folder_path (str): Path to the folder containing documents
        
    Returns:
        List of documents loaded from the folder
    """
    # Create SimpleDirectoryReader instance
    reader = SimpleDirectoryReader(input_dir=folder_path, recursive=True)
    
    # Load and return documents
    documents = reader.load_data()
    
    return documents

# Test the function after you complete it
test_folder = "../data"
documents = load_documents_from_folder(test_folder)
print(f"Loaded {len(documents)} documents")


100%|███████████████████████████████████████| 139M/139M [01:58<00:00, 1.23MiB/s]


Failed to load file c:\Users\gengi\OneDrive\Desktop\ai-accelerator-C2\Day_6\session_2\assignments\..\data\audio\ai_agents.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...
Failed to load file c:\Users\gengi\OneDrive\Desktop\ai-accelerator-C2\Day_6\session_2\assignments\..\data\audio\in_the_end.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...


c:\python\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Failed to load file c:\Users\gengi\OneDrive\Desktop\ai-accelerator-C2\Day_6\session_2\assignments\..\data\audio\rags.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...
Loaded 39 documents


## 2. Vector Store Creation Function

Complete the function below to create a LanceDB vector store.


In [5]:
def create_vector_store(db_path: str = "./vectordb", table_name: str = "documents"):
    """
    Create a LanceDB vector store for storing document embeddings.
    
    TODO: Complete this function to create and configure a LanceDB vector store.
    HINT: Use LanceDBVectorStore with uri and table_name parameters
    
    Args:
        db_path (str): Path where the vector database will be stored
        table_name (str): Name of the table in the vector database
        
    Returns:
        LanceDBVectorStore: Configured vector store
    """
    # Create the directory if it doesn't exist
    Path(db_path).mkdir(parents=True, exist_ok=True)
    
    # Create vector store
    vector_store = LanceDBVectorStore(uri=db_path, table_name=table_name)
    
    return vector_store

# Test the function after you complete it
vector_store = create_vector_store("./assignment_vectordb")
print(f"Vector store created: {vector_store is not None}")


2025-11-02 13:52:16,133 - WARNING - Table documents doesn't exist yet. Please add some data to create it.


Vector store created: True


## 3. Vector Index Creation Function

Complete the function below to create a vector index from documents.


In [6]:
def create_vector_index(documents: List, vector_store):
    """
    Create a vector index from documents using the provided vector store.
    
    TODO: Complete this function to create a VectorStoreIndex from documents.
    HINT: Create StorageContext with vector_store, then use VectorStoreIndex.from_documents()
    
    Args:
        documents: List of documents to index
        vector_store: LanceDB vector store to use for storage
        
    Returns:
        VectorStoreIndex: The created vector index
    """
    # Create storage context with vector store
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    # Create index from documents
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        show_progress=True
    )
    
    return index

# Test the function after you complete it (will only work after previous functions are completed)
if documents and vector_store:
    index = create_vector_index(documents, vector_store)
    print(f"Vector index created: {index is not None}")
else:
    print("Complete previous functions first to test this one")


Generating embeddings: 100%|██████████| 52/52 [00:20<00:00,  2.53it/s]
2025-11-02 13:52:56,491 - INFO - Create new table documents adding data.


Vector index created: True


## 4. Document Search Function

Complete the function below to search for relevant documents using the vector index.


In [7]:
def search_documents(index, query: str, top_k: int = 3):
    """
    Search for relevant documents using the vector index.
    
    TODO: Complete this function to perform semantic search on the index.
    HINT: Use index.as_retriever() with similarity_top_k parameter, then retrieve(query)
    
    Args:
        index: Vector index to search
        query (str): Search query
        top_k (int): Number of top results to return
        
    Returns:
        List of retrieved document nodes
    """
    # Create retriever from index
    retriever = index.as_retriever(similarity_top_k=top_k)
    
    # Retrieve documents for the query
    results = retriever.retrieve(query)
    
    return results

# Test the function after you complete it (will only work after all previous functions are completed)
if 'index' in locals() and index is not None:
    test_query = "What are AI agents?"
    results = search_documents(index, test_query, top_k=2)
    print(f"Found {len(results)} results for query: '{test_query}'")
    for i, result in enumerate(results, 1):
        print(f"Result {i}: {result.text[:100] if hasattr(result, 'text') else 'No text'}...")
else:
    print("Complete all previous functions first to test this one")


2025-11-02 13:53:10,611 - INFO - query_type :, vector


Found 2 results for query: 'What are AI agents?'
Result 1: THE LANDSCAPE OF EMERGING AI AGENT ARCHITECTURES
FOR REASONING , PLANNING , AND TOOL CALLING : A S U...
Result 2: agent-personas or the user is not needed, multi-agent architectures tend to thrive more when collabo...


## 5. Final Test - Complete Pipeline

Once you've completed all the functions above, run this cell to test the complete pipeline with multiple search queries.


In [8]:
# Final test of the complete pipeline
print("🚀 Testing Complete Vector Database Pipeline")
print("=" * 50)

# Re-run the complete pipeline to ensure everything works
data_folder = "../data"
vector_db_path = "./assignment_vectordb"

# Step 1: Load documents
print("\n📂 Step 1: Loading documents...")
documents = load_documents_from_folder(data_folder)
print(f"   Loaded {len(documents)} documents")

# Step 2: Create vector store
print("\n🗄️ Step 2: Creating vector store...")
vector_store = create_vector_store(vector_db_path)
print("   Vector store status:", "✅ Created" if vector_store else "❌ Failed")

# Step 3: Create vector index
print("\n🔗 Step 3: Creating vector index...")
if documents and vector_store:
    index = create_vector_index(documents, vector_store)
    print("   Index status:", "✅ Created" if index else "❌ Failed")
else:
    index = None
    print("   ❌ Cannot create index - missing documents or vector store")

# Step 4: Test multiple search queries
print("\n🔍 Step 4: Testing search functionality...")
if index:
    search_queries = [
        "What are AI agents?",
        "How to evaluate agent performance?", 
        "Italian recipes and cooking",
        "Financial analysis and investment"
    ]
    
    for query in search_queries:
        print(f"\n   🔎 Query: '{query}'")
        results = search_documents(index, query, top_k=2)
        
        if results:
            for i, result in enumerate(results, 1):
                text_preview = result.text[:100] if hasattr(result, 'text') else "No text available"
                score = f" (Score: {result.score:.4f})" if hasattr(result, 'score') else ""
                print(f"      {i}. {text_preview}...{score}")
        else:
            print("      No results found")
else:
    print("   ❌ Cannot test search - index not created")

print("\n" + "=" * 50)
print("🎯 Assignment Status:")
print(f"   Documents loaded: {'✅' if documents else '❌'}")
print(f"   Vector store created: {'✅' if vector_store else '❌'}")
print(f"   Index created: {'✅' if index else '❌'}")
print(f"   Search working: {'✅' if index else '❌'}")

if documents and vector_store and index:
    print("\n🎉 Congratulations! You've successfully completed the assignment!")
    print("   You've built a complete vector database with search functionality!")
else:
    print("\n📝 Please complete the TODO functions above to finish the assignment.")


🚀 Testing Complete Vector Database Pipeline

📂 Step 1: Loading documents...
Failed to load file c:\Users\gengi\OneDrive\Desktop\ai-accelerator-C2\Day_6\session_2\assignments\..\data\audio\ai_agents.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...
Failed to load file c:\Users\gengi\OneDrive\Desktop\ai-accelerator-C2\Day_6\session_2\assignments\..\data\audio\in_the_end.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...
Failed to load file c:\Users\gengi\OneDrive\Desktop\ai-accelerator-C2\Day_6\session_2\assignments\..\data\audio\rags.mp3 with error: [WinError 2] The system cannot find the file specified. Skipping...


c:\python\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


   Loaded 39 documents

🗄️ Step 2: Creating vector store...
   Vector store status: ✅ Created

🔗 Step 3: Creating vector index...


Generating embeddings: 100%|██████████| 52/52 [00:17<00:00,  3.01it/s]
2025-11-02 13:53:50,179 - INFO - query_type :, vector
2025-11-02 13:53:50,261 - INFO - query_type :, vector
2025-11-02 13:53:50,330 - INFO - query_type :, vector


   Index status: ✅ Created

🔍 Step 4: Testing search functionality...

   🔎 Query: 'What are AI agents?'
      1. THE LANDSCAPE OF EMERGING AI AGENT ARCHITECTURES
FOR REASONING , PLANNING , AND TOOL CALLING : A S U... (Score: 0.6220)

   🔎 Query: 'How to evaluate agent performance?'
      1. steps, but the answers are limited to Yes/No responses [7]. As the industry continues to pivot towar... (Score: 0.6772)

   🔎 Query: 'Italian recipes and cooking'


2025-11-02 13:53:50,406 - INFO - query_type :, vector


      1. # 🍝 Classic Spaghetti Carbonara Recipe

## Ingredients
- 400g spaghetti pasta
- 4 large egg yolks
- ... (Score: 0.6188)

   🔎 Query: 'Financial analysis and investment'
      1. However, several important considerations need to be ad-
dressed in future work:
• Scalability: Eval... (Score: 0.5613)

🎯 Assignment Status:
   Documents loaded: ✅
   Vector store created: ✅
   Index created: ✅
   Search working: ✅

🎉 Congratulations! You've successfully completed the assignment!
   You've built a complete vector database with search functionality!
